In [ ]:
using JuMP
using Gurobi
using CSV
using DataFrames
using PowerModels
using Ipopt
using SparseArrays


const GRB_ENV = Gurobi.Env(output_flag=0);

In [ ]:
#Parse the .m file to extract the data
network_data = PowerModels.parse_file("case18.m")


In [100]:
function convert_to_mat(super_keys, super_key_num, network_data)
    name_keys = collect(keys(network_data[super_keys[super_key_num]]))
    #Collect all parameters used by each entry
    info_keys = collect(keys(network_data[super_keys[super_key_num]][name_keys[1]]))
    
    #reconstruct matrix
    matrix = zeros(length(name_keys), length(info_keys))
    for i in 1:length(name_keys)
        for j in 1:length(info_keys)
            #source_id returns a vector? weird, IDK why
            if info_keys[j] != "source_id"
                val = network_data[super_keys[super_key_num]][name_keys[i]][info_keys[j]]
            else 
                val = 0
            end
            matrix[i,j] = val
        end
    end
    return name_keys, info_keys, matrix
end

convert_to_mat (generic function with 1 method)

In [153]:
function reorder_mat_from_name_key(matrix, sort_ref)
    sort_ref_num = zeros(1, length(sort_ref))
    for i = 1:length(sort_ref)
        sort_ref_num[i] = parse(Int, sort_ref[i])
    end
    indeces = sortperm(sort_ref_num[1,:])
    #Reorder the branches based on their names
    new_matrix = matrix[indeces, :]
    return new_matrix
end

reorder_mat_from_name_key (generic function with 1 method)

In [154]:
#Example: network_data["branch"]["1"]["br_r"] this reads the branch resistance of the first branch

#Generate the list of all keys in the network data
# "bus"
# "source_type"
# "name"
# "dcline"
# "source_version"
# "gen"
# "branch"
# "storage"
# "switch"
# "baseMVA"
# "per_unit"
# "shunt"
# "load"
super_keys = collect(keys(network_data))

#Collect the keys for the bus data, gives the name of every bus
bus_key_num = 1;
bus_name_keys, bus_info_keys, bus_matrix = convert_to_mat(super_keys, bus_key_num, network_data)
bus_matrix = reorder_mat_from_name_key(bus_matrix, bus_name_keys)

#Reconstruct branch data matrix
branch_key_num = 7;
branch_name_keys, branch_info_keys, branch_matrix = convert_to_mat(super_keys, branch_key_num, network_data)
branch_matrix = reorder_mat_from_name_key(branch_matrix, branch_name_keys)

#Reconstruct the load data matrix
load_key_num = 13;
load_name_keys, load_info_keys, load_matrix = convert_to_mat(super_keys, load_key_num, network_data)
load_matrix = reorder_mat_from_name_key(load_matrix, load_name_keys)


15×6 Matrix{Float64}:
 0.0   2.0  1.0  0.012  0.02   1.0
 0.0   3.0  1.0  0.025  0.04   2.0
 0.0   4.0  1.0  0.093  0.15   3.0
 0.0   5.0  1.0  0.226  0.3    4.0
 0.0   6.0  1.0  0.05   0.08   5.0
 0.0   7.0  1.0  0.012  0.02   6.0
 0.0   8.0  1.0  0.062  0.1    7.0
 0.0   9.0  1.0  0.031  0.05   8.0
 0.0  20.0  1.0  0.062  0.1    9.0
 0.0  21.0  1.0  0.019  0.03  10.0
 0.0  22.0  1.0  0.012  0.02  11.0
 0.0  23.0  1.0  0.05   0.08  12.0
 0.0  24.0  1.0  0.031  0.05  13.0
 0.0  25.0  1.0  0.062  0.1   14.0
 0.0  26.0  1.0  0.012  0.02  15.0

In [158]:
results = PowerModels.compute_ac_pf(network_data)

Dict{String, Any} with 5 entries:
  "optimizer"          => "NLsolve"
  "termination_status" => true
  "objective"          => 0.0
  "solution"           => Dict{String, Any}("gen"=>Dict{String, Any}("1"=>Dict(…
  "solve_time"         => 2.173

In [181]:
AdmitMat = PowerModels.calc_admittance_matrix(network_data)
#This gives the network admittance matrix in complex rectangular form
AdmitMat.matrix

0.0 + 0.0im